
## Word Embeding
### 🚨 1º - Dar Upload no colab 
- Configuration of GPU mode:
> Primeiro ir à **tab** Runtime -> Change runtime type -> Hardware accelarator = GPU


> Necessary Imports

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import confusion_matrix

## Load Data

> Nota: Colocar tudo na drive como zip, pois é mais rápido !

In [ ]:
#Colocar os dados na drive
from google.colab import drive
drive.mount('/content/drive')
#!unzip "/content/drive/My Drive/LEI/comments_processed.json.zip" 
!unzip "/content/drive/My Drive/LEI/glove.6B.zip" 
!unzip "/content/drive/My Drive/LEI/featherized.ftr.zip"
!unzip "/content/drive/My Drive/LEI/beast_model_m.keras.zip"

In [ ]:
import json
import pandas as pd
#load data into arrays
#
def load_Data(filename):
  d_polarity = {'negative':0,'neutro':1, 'positive':2}
  samples_0=[]
  samples_1=[]
  samples_2=[]
  aux=False
  if filename == 'featherized.ftr':
    data = pd.read_feather('featherized.ftr', columns=None, use_threads=True)
    aux=True
  else:
    f = open('comments_processed.json','r')
    data=json.load(f)

  #separate data by polarity
  if aux:
    for i,row in data.iterrows():
      if row['polarityClass'] == 0:
        samples_0.append(row['comentario']) #CHAMAR A FUNÇÂÂO DO PREPROCESSAMENTO 
      elif row['polarityClass']  == 1:
        samples_1.append(row['comentario'])
      elif row['polarityClass']  == 2:
        samples_2.append(row['comentario'])
        
  else:
    for i in data:
      if d_polarity[data[i]['rev_polarity']] == 0:
        samples_0.append(data[i]['rev_text']) #CHAMAR A FUNÇÂÂO DO PREPROCESSAMENTO 
      elif d_polarity[data[i]['rev_polarity']] == 1:
        samples_1.append(data[i]['rev_text']) 
      elif d_polarity[data[i]['rev_polarity']] == 2:
        samples_2.append(data[i]['rev_text'])

  
  len_0=len(samples_0)
  len_1=len(samples_1)
  len_2=len(samples_2)
  
  #create 3 datasets for train, validation and test (0.6 Train, 0.2 Validation, 0.2 Test)
  
  train_samples = samples_0[:int(0.6*len_0)]           + samples_1[:int(0.6*len_1)]          + samples_2[:int(0.6*len_2)]
  train_labels= [0]* int(0.6*len_0)                    + [1]*int(0.6*len_1)                   + [2] * int(0.6*len_2)
  
  val_samples  =  samples_0[int(0.6*len_0):int(0.8*len_0)]  + samples_1[int(0.6*len_1):int(0.8*len_1)] + samples_2[int(0.6*len_2):int(0.8*len_2)] 
  val_labels = [0]*(int(0.8*len_0) - int(0.6*len_0))        + [1]*(int(0.8*len_1) - int(0.6*len_1))    + [2] * (int(0.8*len_2) - int(0.6*len_2)) 
  
  test_samples =  samples_0[int(0.8*len_0):]           +  samples_1[int(0.8*len_1):]         + samples_2[int(0.8*len_2):]
  test_labels = [0] * (len_0 - int(0.8*len_0))         + [1] * (len_1 - int(0.8*len_1))       + [2] * (len_2 - int(0.8*len_2))       
  
  
  #shuffle data
  
  import random
  c = list(zip(train_samples, train_labels))
  random.shuffle(c)
  train_samples, train_labels = zip(*c)
  
  c = list(zip(val_samples, val_labels))
  random.shuffle(c)
  val_samples, val_labels = zip(*c)
  
  c = list(zip(test_samples, test_labels))
  random.shuffle(c)
  test_samples, test_labels = zip(*c)
  return train_samples, train_labels, val_samples, val_labels,test_samples, test_labels
train_samples, train_labels, val_samples, val_labels,test_samples, test_labels=load_Data('featherized.ftr')



In [ ]:
'''
#verificaçãão dos dados.
print(train_samples[0])
print(train_labels[0])
for s in samples_0:
  if s==train_samples[0]:
    print("YES_0")
    print("\t "+ s )
      
for s in samples_1:
  if s==train_samples[0]:
    print("YES_1")
    print("\t "+ s )

for s in samples_2:
  if s==train_samples[0]:
    print("YES_2")
    print("\t "+ s )
      
'''   

## Create a vocabulary index

> Our layer will only consider the top 20,000 words, and will truncate or pad sequences to be actually 200 tokens long.

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import pickle
vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices([str(x) for x in train_samples]).batch(128)#.batch(128)
vectorizer.adapt(text_ds) # give context to the network

#Dump vectorizer to a file 
#pickle.dump({'config': vectorizer.get_config(),'weights': vectorizer.get_weights()}, open("/content/drive/My Drive/tv_layer.pkl", "wb"))

voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

> Donwload Global Vectors for Word Representation from site.<br/>

>Note: It can be stored in the drive for faster loading. The code is in the load data section above.

In [ ]:
#Donwload pre trained model
#https://nlp.stanford.edu/projects/glove/
#!wget http://nlp.stanford.edu/data/glove.6B.zip
#!unzip -q glove.6B.zip


> Load vector from file and create embedding matrix

In [ ]:
path_to_glove_file ="glove.6B.300d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

In [ ]:
num_tokens = len(voc) + 2
embedding_dim = 300
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))


> Plot the words

In [ ]:
#plot words 
#https://towardsdatascience.com/getting-started-with-text-analysis-in-python-ca13590eb4f7
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
words =  list(embeddings_index.keys())
vectors = [embeddings_index[word] for word in words]
Y = tsne.fit_transform(vectors[:1000])
plt.scatter(Y[:, 0], Y[:, 1])

for label, x, y in zip(words, Y[:, 0], Y[:, 1]):
    plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords="offset points")
plt.show()

> Processing data for glove embeding layer

In [ ]:
#prepare data for CNN
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

#y_train = np.array(train_labels,dtype=np.float32)
#y_val = np.array(val_labels,dtype=np.float32)
y_train = np.array(train_labels)
y_val = np.array(val_labels)

In [ ]:
#plot data
unique_elements, counts_elements = np.unique(y_train, return_counts=True)
bar=sns.barplot(unique_elements,counts_elements, palette='mako')

## Balance Dataset

In [ ]:
#!pip install imblearn
#!pip install imbalanced-learn

In [ ]:
%%time

import imblearn
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks

#Estrategias para fazer under e over sampling
strategy = {0:7000,1:7000} # over na classe 0 e 1.
strategy2 = {0:7000,1:7000,2:7000} #under nas 3 classes.
#sm = SMOTE("minority")#(random_state = 2,sampling_strategy=strategy)
over = RandomOverSampler(sampling_strategy=strategy)
#under = TomekLinks()
under = RandomUnderSampler(sampling_strategy=strategy2)
XX,YY=over.fit_sample(x_train,y_train)
X,Y=under.fit_resample(XX,YY)





In [ ]:
#print dataset
unique_elements, counts_elements = np.unique(Y, return_counts=True)
bar=sns.barplot(unique_elements,counts_elements, palette='mako')


> **CallBacks** for Models<br/>
> Available callbacks on  https://keras.io/api/callbacks/

In [ ]:

filename="weights_best.h5"
cb2 = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
cb3 = tf.keras.callbacks.ModelCheckpoint(
    filepath= filename,
    save_weights_only=True,
    monitor='val_loss',#val_loss',#'val_mean_squared_error',
    mode='min',
    save_best_only= True)

cb4 = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3) #se apartir de 2 epochs a loss não descer,o learning rate diminui em 10%.
my_callbacks=[
    cb3#,cb2#,cb4
]

##1º Model- CNN 
> The Functional API 

In [ ]:
from tensorflow.keras import layers


embedding_layer = layers.Embedding(num_tokens,embedding_dim,
                            embeddings_initializer = keras.initializers.Constant(embedding_matrix),
                            trainable=False,)

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(416, 1, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D()(x)
x = layers.Conv1D(496, 1, activation="relu")(x)
x = layers.MaxPooling1D()(x)
x = layers.Conv1D(320, 1, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)

x = layers.Flatten()(x)
x = layers.Dense(64, activation="relu")(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(64, activation="relu")(x)

preds = layers.Dense(3, activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)
#model.summary()




> Run Model

In [ ]:
model.compile(
    optimizer='Adamax',
    loss="sparse_categorical_crossentropy", 
    metrics=["accuracy"]
)
model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_val, y_val),  callbacks=my_callbacks )

> For **see the results**, go to section "Predict data and Visualization of results"

## 2º Model - RNN 
> LSTM layres



In [ ]:
embedding_layer = layers.Embedding(num_tokens,embedding_dim,
                  embeddings_initializer = keras.initializers.Constant(embedding_matrix),
                  trainable=False,)

model = keras.Sequential([
      embedding_layer,
      #keras.layers.Bidirectional(tf.keras.layers.LSTM(128,activation='tanh',return_sequences=True,dropout=0.1)),
      layers.LSTM(128,activation='tanh',return_sequences=True,dropout=0.1),
      layers.GlobalMaxPooling1D(),
      layers.Dense(
        units=64,
        activation='relu'
      ),
      layers.Dropout(0.1),
      layers.Dense(
        units=64,
        activation='relu'
      ),
      layers.Dense(3, activation= 'softmax')
])
#model.summary()

> Run Model

In [ ]:
model.compile(optimizer='Adamax',#hp.Choice('Optimizer', values = optimizer), #1e-2,
              loss='sparse_categorical_crossentropy',#tf.keras.losses.CategoricalCrossentropy()
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_val, y_val), callbacks=my_callbacks)

> For **see the results**, go to section "Predict data and Visualization of results"

## Keras Tuner
> Calculate the best parameters for the networks

In [ ]:
!pip install keras-tuner

In [ ]:
#genes
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']

In [ ]:
from tensorflow.keras import layers
def build_model(hp):  
    embedding_layer = layers.Embedding(num_tokens,embedding_dim,
                            embeddings_initializer = keras.initializers.Constant(embedding_matrix),
                            trainable=False,)

#    int_sequences_input = keras.Input(shape=(None,), dtype="int64")
#    embedded_sequences = embedding_layer(int_sequences_input)
    ker_size = hp.Choice('kernel_Size', values = [1,3]),
    model = keras.Sequential([
      embedding_layer,
      layers.Conv1D(hp.Int('Conv_1_filter', min_value=128, max_value=512, step=16),ker_size , activation="relu"),
      layers.MaxPooling1D(),
      layers.Conv1D(hp.Int('Conv_2_filter', min_value=128, max_value=512, step=16), ker_size, activation="relu"),
      layers.MaxPooling1D(),
      layers.Dropout(0.1),
      layers.Conv1D(hp.Int('Conv_3_filter', min_value=128, max_value=512, step=16), ker_size, activation="relu"),
      #layers.MaxPooling1D(),
      
      layers.Dropout(0.2),
      layers.LSTM(128,return_sequences=True,dropout=0.1),   
      layers.GlobalMaxPooling1D(),

      layers.Dense(
         units=64,#hp.Int('Dense_1_units', min_value=128, max_value=1024, step=16),
        activation='relu'
      ),
      layers.Dropout(0.1),
      layers.Dense(
        units=64,#hp.Int('Dense_1_units', min_value=128, max_value=1024, step=16),
        activation='relu'
      ),
      layers.Dense(3, activation= 'softmax')#
    ])
  
    model.compile(optimizer='Adamax',#hp.Choice('Optimizer', values = optimizer), #1e-2,
              loss='sparse_categorical_crossentropy',#tf.keras.losses.CategoricalCrossentropy()
              metrics=['accuracy'])
  
    return model
  #optimizer='rmsprop'

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

tuner_search=RandomSearch(build_model,
                          #objective='val_accuracy',
                          objective='val_loss',
                          max_trials=20,directory='output',project_name="WE")


tuner_search.search(x_train,y_train,validation_data=(x_val, y_val),epochs=10,callbacks=my_callbacks)
#Best parameters
best_hyperparameters = tuner_search.get_best_hyperparameters(1)[0]
print(best_hyperparameters.values)
#get best model
model=tuner_search.get_best_models(num_models=1)[0]
model.summary()

> Find the best model in tunner

In [ ]:
#get best model
model=tuner_search.get_best_models(num_models=1)[0]
best_hyperparameters = tuner_search.get_best_hyperparameters(1)[0]
print(best_hyperparameters.values)
model.summary()
#model.compile(
#    loss='sparse_categorical_crossentropy',#[categorical_focal_loss(alpha=[[.25, .25, .25]], gamma=2)], 
#    optimizer=tf.keras.optimizers.Adagrad() , metrics=["accuracy"]
#)
#model.fit(x_train, y_train, batch_size=64, epochs=15, validation_data=(x_val, y_val),   callbacks=my_callbacks)

## Best Model - CRNN 
> CNN layers and RNN layers (MIXED)
- Verificar a métrica **mcc metric**


In [ ]:
from tensorflow.keras import layers
embedding_layer = layers.Embedding(num_tokens,embedding_dim,
                  embeddings_initializer = keras.initializers.Constant(embedding_matrix),
                  trainable=False,)

model = keras.Sequential([
      embedding_layer,
      layers.Conv1D(416, 1, activation="relu"),
      layers.MaxPooling1D(),
      layers.Conv1D(496, 1, activation="relu"),
      layers.MaxPooling1D(), 
      layers.Dropout(0.1),
      layers.Conv1D(320, 1, activation="relu"),
      layers.MaxPooling1D(),

      layers.LSTM(128, activation='tanh',return_sequences=True,dropout=0.1),
      #layers.BatchNormalization(),
      #layers.LayerNormalization(),
      layers.GlobalMaxPooling1D(),
      layers.Dense(units=64,activation='relu'),
      layers.Dropout(0.1),
      layers.Dense(units=64,activation='relu'),

      layers.Dense(3, activation= 'softmax')
])
#model.summary()

> Run Model

In [ ]:
model.compile(optimizer='Adamax',#'Adam','rmsprop'
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])#MeanSquaredError
              
model.fit(X, Y, batch_size=64, epochs=10, validation_data=(x_val, y_val), callbacks=my_callbacks)

> Save the model

In [ ]:
name_best_model ="best_model_1.keras"
model.save(name_best_model)

In [ ]:
#COPY TO DRIVE
#!cp name_best_model "/content/drive/My Drive/"

## Predict data and Visualization of results


> https://stats.stackexchange.com/questions/276067/whats-considered-a-good-log-loss

In [ ]:
#Load bestmodel for Hybrid model 
#model.load_weights("beast_model_m.keras")

In [ ]:
#vetorizer de dataset test
x_test = vectorizer(np.array([[s] for s in test_samples])).numpy()
y_test = np.array(test_labels)

#LOAD BEST EPOCH
#model.load_weights(filename)#load best epoch
#model.evaluate(x_test,y_test) 

#model predict
preds=np.round(model.predict(x_test),0)
pred_labels = np.argmax(preds, axis=-1)

In [ ]:
#create confusion_matrix on heatmap 
cm = confusion_matrix(np.array(test_labels),np.array(pred_labels))
labels = ['Negative', 'Neutro','Positive'] # labels for y-axis
labels2 = ['P_Negative', 'P_Neutro','P_Positive'] # labels for x-axis
sns.heatmap(#cm, fmt='g',
            cm/np.sum(cm), fmt='.2%', 
            annot=True,
            cmap='Blues',xticklabels=labels2, yticklabels=labels)

#report for metrics
cm=metrics.classification_report(np.array(test_labels),np.array(pred_labels),target_names=["Negative","Neutro","Positive"])
print(cm)